In [ ]:
import torch
import random
import numpy as np

from tqdm import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)

from datasets import Dataset
from typing import List , Optional

: 

Basically works by:
- LeanDojo extracts state-tactic pairs from mathlib